# ノイズ検出モデルの学習（Google Colab版）

このノートブックは、30クラス分類によるノイズ検出モデルの学習を行います。

## 📋 セットアップ手順

1. **GPUを有効化**: ランタイム → ランタイムのタイプを変更 → GPU（T4）を選択
2. **Google Driveをマウント**（推奨）:
   - プロジェクトフォルダをGoogle Driveにアップロード
   - セル1でDriveをマウント
3. **データセットの準備**:
   - セル2でデータセットを生成（`baseline_dataset.pickle`）
   - ノイズタイプ: `frequency_band`（周波数帯域集中ノイズ）
   - 区間数: 30区間
4. 各セルを順番に実行してください

## 📝 ノイズタイプについて

現在使用しているノイズタイプ:
- **`frequency_band`**: 周波数帯域集中ノイズ（特定の周波数帯域に集中的に発生）
  - 電源ノイズ、共振、クロストークなどを模擬
  - 区間内の特定の周波数帯域（30%程度）にガウシアン分布でノイズが集中


In [ ]:
# Google Driveをマウント（推奨）
from google.colab import drive
drive.mount('/content/drive')

# 作業ディレクトリを変更（プロジェクトフォルダに移動）
import os
os.chdir('/content/drive/MyDrive/noise')  # プロジェクトフォルダのパスに変更してください

print(f"現在の作業ディレクトリ: {os.getcwd()}")
print(f"ファイル一覧: {os.listdir('.')[:10]}")  # 最初の10ファイルを表示


## 📦 ライブラリのインストールとインポート


In [ ]:
# ライブラリのインストールとインポート
!pip install torch torchvision scikit-learn matplotlib -q

import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import platform
import os
import sys
import time
import warnings

# matplotlibのキャッシュをクリア（フォント警告を防ぐため）
import shutil
try:
    shutil.rmtree('/root/.cache/matplotlib', ignore_errors=True)
except:
    pass

# matplotlibの警告を抑制
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')

# プロジェクトルートをパスに追加
project_root = os.getcwd()
sys.path.insert(0, project_root)
sys.path.insert(0, os.path.join(project_root, '畳み込み'))
sys.path.insert(0, os.path.join(project_root, 'ノイズの付与(共通)'))

# GPUの確認
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用デバイス: {device}")
if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")
    print(f"GPUメモリ: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# 日本語フォントの設定（Colab用）
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False


In [ ]:
# データセットの準備
# 畳み込み/dataset.py を実行してデータセットを生成
print("データセットを準備中...")
print("=" * 60)

# プロジェクトルートとnoiseモジュールのパスを追加
project_root = os.getcwd()
noise_folder_path = os.path.join(project_root, 'ノイズの付与(共通)')
noise_link_path = os.path.join(project_root, 'noise')

# プロジェクトルートをsys.pathに追加（dataset.pyの中でproject_rootからnoiseをインポートするため）
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# ノイズの付与(共通)フォルダをnoiseという名前でシンボリックリンクを作成
# （既に存在する場合はスキップ）
if not os.path.exists(noise_link_path):
    try:
        os.symlink(noise_folder_path, noise_link_path)
        print(f"シンボリックリンクを作成: {noise_link_path} -> {noise_folder_path}")
    except Exception as e:
        print(f"シンボリックリンクの作成に失敗: {e}")
        print("代わりにsys.modulesに直接登録します...")
        # シンボリックリンクが作成できない場合の代替方法
        import importlib.util
        noise_spec = importlib.util.spec_from_file_location("noise", os.path.join(noise_folder_path, "__init__.py"))
        noise_module = importlib.util.module_from_spec(noise_spec)
        sys.modules["noise"] = noise_module
        noise_spec.loader.exec_module(noise_module)
        
        add_noise_spec = importlib.util.spec_from_file_location("noise.add_noise", os.path.join(noise_folder_path, "add_noise.py"))
        add_noise_module = importlib.util.module_from_spec(add_noise_spec)
        sys.modules["noise.add_noise"] = add_noise_module
        add_noise_spec.loader.exec_module(add_noise_module)
else:
    print(f"シンボリックリンクは既に存在します: {noise_link_path}")

# __file__を定義（exec()で実行する場合に必要）
dataset_path = os.path.join(project_root, '畳み込み', 'dataset.py')

# グローバル名前空間に__file__を追加して実行
# exec()の第2引数にglobals()を渡すと、実行されたコードの変数が現在の名前空間に反映される
exec_globals = globals()
exec_globals['__file__'] = dataset_path

# dataset.pyを実行
exec(open(dataset_path).read(), exec_globals)

print("\n✅ データセット準備完了！")
print("   baseline_dataset.pickle が生成されました")


In [ ]:
# モデルと評価関数をインポート
from 畳み込み.model import SimpleCNN
from eval import evaluate_baseline_model

print("✓ モジュールのインポート完了")


In [ ]:
# データセットクラス
class PSDDataset(Dataset):
    """PSDデータセット"""
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


In [ ]:
def validate(model, dataloader, criterion, device):
    """検証"""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for data, labels in dataloader:
            data = data.to(device)
            labels = labels.to(device)
            
            outputs = model(data)
            
            # NaN/Infチェック
            if torch.isnan(outputs).any() or torch.isinf(outputs).any():
                print(f"警告: モデル出力にNaN/Infが検出されました")
                print(f"  NaN: {torch.isnan(outputs).sum().item()}, Inf: {torch.isinf(outputs).sum().item()}")
                continue
            
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(dataloader)
    accuracy = 100 * correct / total
    
    return avg_loss, accuracy, all_predictions, all_labels


In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    """1エポックの学習"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for data, labels in dataloader:
        data = data.to(device)
        labels = labels.to(device)
        
        # 順伝播
        optimizer.zero_grad()
        outputs = model(data)
        
        # NaN/Infチェック
        if torch.isnan(outputs).any() or torch.isinf(outputs).any():
            print(f"警告: モデル出力にNaN/Infが検出されました")
            print(f"  NaN: {torch.isnan(outputs).sum().item()}, Inf: {torch.isinf(outputs).sum().item()}")
            continue
        
        loss = criterion(outputs, labels)
        
        # NaN/Infチェック
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"警告: 損失にNaN/Infが検出されました")
            continue
        
        # 逆伝播
        loss.backward()
        
        # 勾配クリッピング（勾配爆発を防ぐ）
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        # 統計
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(dataloader)
    accuracy = 100 * correct / total
    
    return avg_loss, accuracy


In [ ]:
# ハイパーパラメータ（データセット読み込み前に定義）
NUM_CLASSES = 30  # 30クラス分類

# データセットの読み込み
print("データセットを読み込み中...")
with open('baseline_dataset.pickle', 'rb') as f:
    dataset = pickle.load(f)

train_data = dataset['train']['data']
train_labels = dataset['train']['labels']
val_data = dataset['val']['data']
val_labels = dataset['val']['labels']
test_data = dataset['test']['data']
test_labels = dataset['test']['labels']

print(f"訓練データ: {len(train_data):,}サンプル")
print(f"検証データ: {len(val_data):,}サンプル")
print(f"テストデータ: {len(test_data):,}サンプル")

# データをテンソルに変換
train_data = torch.FloatTensor(train_data)
train_labels = torch.LongTensor(train_labels)
val_data = torch.FloatTensor(val_data)
val_labels = torch.LongTensor(val_labels)
test_data = torch.FloatTensor(test_data)
test_labels = torch.LongTensor(test_labels)

# ラベルの確認（30クラス分類用）
print(f"\nラベルの確認:")
print(f"  訓練データのラベル範囲: {train_labels.min().item()} - {train_labels.max().item()}")
print(f"  期待される範囲: 0 - {NUM_CLASSES-1}")
if train_labels.max().item() >= NUM_CLASSES:
    raise ValueError(f"エラー: ラベルの最大値({train_labels.max().item()})がNUM_CLASSES({NUM_CLASSES})以上です。")
print("✓ ラベルが正しい範囲内です")

# データの前処理確認（dataset.pyで既に前処理済み）
print("\nデータの前処理確認:")
print(f"  訓練データの平均: {train_data.mean():.6f}")
print(f"  訓練データの標準偏差: {train_data.std():.6f}")
print(f"  訓練データの範囲: [{train_data.min():.6f}, {train_data.max():.6f}]")
print("✓ データは既に前処理済みです（dataset.pyでスケーリング + ログ変換 + 正規化済み）")
print("✓ 追加の前処理は不要です")


In [ ]:
# ハイパーパラメータ
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
NUM_CLASSES = 30  # 30クラス分類

# モデルの作成
model = SimpleCNN(num_classes=NUM_CLASSES).to(device)
print(f"モデルパラメータ数: {sum(p.numel() for p in model.parameters()):,}")

# DataLoaderの作成
train_dataset = PSDDataset(train_data, train_labels)
val_dataset = PSDDataset(val_data, val_labels)
test_dataset = PSDDataset(test_data, test_labels)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# クラス重みの計算（クラス不均衡に対応）
# クラス重みの計算（クラス不均衡に対応）
print("\nクラス重みを計算中...")
print(f"ラベルの範囲: {train_labels.min().item()} - {train_labels.max().item()}")
print(f"NUM_CLASSES: {NUM_CLASSES}")
# ラベルがNUM_CLASSESを超えていないか確認
if train_labels.max().item() >= NUM_CLASSES:
    raise ValueError(f"エラー: ラベルの最大値({train_labels.max().item()})がNUM_CLASSES({NUM_CLASSES})以上です。データセットを10クラス用に再生成してください。")
# クラス重みを計算（確実にNUM_CLASSES要素にする）
label_counts_full = torch.bincount(train_labels)
label_counts = torch.zeros(NUM_CLASSES, dtype=torch.long)
for i in range(min(NUM_CLASSES, len(label_counts_full))):
    label_counts[i] = label_counts_full[i]
# 重み = 総サンプル数 / (クラス数 * 各クラスのサンプル数)
class_weights = len(train_labels) / (NUM_CLASSES * (label_counts.float() + 1e-8))
class_weights = class_weights / class_weights.sum() * NUM_CLASSES  # 正規化
print(f"クラス重みの形状: {class_weights.shape}")
print(f"クラス重みの範囲: [{class_weights.min():.3f}, {class_weights.max():.3f}]")

criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)  # 重み減衰を追加
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

print("\n✓ モデルと学習設定の準備完了")


In [ ]:
# 学習実行（100エポック）
# 学習履歴
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
best_val_acc = 0
best_model_state = None
patience = 10  # Early Stoppingのpatience
patience_counter = 0  # Early Stoppingのカウンター
best_val_acc = 0

print("学習開始...")
print("=" * 60)
print(f"総エポック数: {NUM_EPOCHS}")

start_time = time.time()

for epoch in range(NUM_EPOCHS):
    epoch_start = time.time()
    
    # 学習
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # 検証
    val_loss, val_acc, _, _ = validate(model, val_loader, criterion, device)
    
    # 学習率の調整
    scheduler.step(val_loss)
    
    # 履歴の保存
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    # ベストモデルの保存
    # ベストモデルの保存とEarly Stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict().copy()
        patience_counter = 0  # ベスト精度更新時にリセット
    else:
        patience_counter += 1  # ベスト精度が更新されなかった
    
    # Early Stoppingチェック
    if patience_counter >= patience:
        print(f"\n⚠ Early Stopping: {patience}エポック連続で改善なし。学習を停止します。")
        print(f"   ベスト検証精度: {best_val_acc:.2f}% (エポック {epoch+1-patience})")
        break
    
    # ログ出力（5エポックごと、または最初と最後）
    # ログ出力（毎エポック、または最初と最後）
    if (epoch + 1) % 1 == 0 or epoch == 0 or (epoch + 1) == NUM_EPOCHS:
        epoch_time = time.time() - epoch_start
        elapsed_time = time.time() - start_time
        remaining_epochs = NUM_EPOCHS - (epoch + 1)
        estimated_remaining = (elapsed_time / (epoch + 1)) * remaining_epochs if epoch > 0 else 0
        
        # 学習率の確認
        current_lr = optimizer.param_groups[0]["lr"]
        
        # モデルの出力を確認（最初のエポックのみ）
        if epoch == 0:
            model.eval()
            with torch.no_grad():
                sample_data = train_data[:1].to(device)
                sample_output = model(sample_data)
                print(f"\n=== エポック {epoch+1} の詳細 ===")
                print(f"  学習率: {current_lr:.6f}")
                print(f"  サンプル出力: {sample_output[0].tolist()}")
                print(f"  予測クラス: {sample_output[0].argmax().item()}")
                print(f"  出力のソフトマックス: {torch.softmax(sample_output[0], dim=0).tolist()}")
            model.train()
        
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] ({epoch_time:.1f}秒)")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"  学習率: {current_lr:.6f}")
        if epoch > 0:
            val_loss_change = val_losses[-1] - val_losses[-2] if len(val_losses) > 1 else 0
            val_acc_change = val_accuracies[-1] - val_accuracies[-2] if len(val_accuracies) > 1 else 0
            print(f"  検証損失の変化: {val_loss_change:+.4f}, 検証精度の変化: {val_acc_change:+.2f}%")
        if estimated_remaining > 0:
            print(f"  経過時間: {elapsed_time/60:.1f}分, 残り見積もり: {estimated_remaining/60:.1f}分")
        print()

total_time = time.time() - start_time

# テストデータで予測結果を取得
print("\nテストデータで予測結果を取得中...")
model.eval()
all_test_predictions = []
all_test_labels = []
all_test_outputs = []

with torch.no_grad():
    for data, labels in test_loader:
        data = data.to(device)
        labels = labels.to(device)
        
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        
        all_test_predictions.extend(predicted.cpu().numpy())
        all_test_labels.extend(labels.cpu().numpy())
        all_test_outputs.extend(torch.softmax(outputs, dim=1).cpu().numpy())

# 予測結果をファイルに保存
import numpy as np
import pandas as pd

results_df = pd.DataFrame({
    "true_label": all_test_labels,
    "predicted_label": all_test_predictions,
    "correct": [int(t == p) for t, p in zip(all_test_labels, all_test_predictions)]
})

# 各クラスの確率も追加
for i in range(10):
    results_df[f"prob_class_{i}"] = [probs[i] for probs in all_test_outputs]

results_df.to_csv("prediction_results.csv", index=False)
print("予測結果を prediction_results.csv に保存しました")

# 混同行列も保存
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(all_test_labels, all_test_predictions)
cm_df = pd.DataFrame(cm, index=[f"True_{i}" for i in range(10)], columns=[f"Pred_{i}" for i in range(10)])
cm_df.to_csv("confusion_matrix.csv")
print("混同行列を confusion_matrix.csv に保存しました")

# 統計情報を表示
print("\n=== 予測結果の統計 ===")
print(f"正解数: {results_df['correct'].sum()} / {len(results_df)}")
print(f"精度: {results_df['correct'].mean()*100:.2f}%")
print("\nクラスごとの正解数:")
for i in range(10):
    class_mask = results_df["true_label"] == i
    correct_count = results_df[class_mask]["correct"].sum()
    total_count = class_mask.sum()
    print(f"  クラス {i}: {correct_count} / {total_count} ({correct_count/total_count*100:.1f}%)")
print(f"学習完了！総時間: {total_time/60:.1f}分 ({total_time/3600:.2f}時間)")


In [ ]:
# ベストモデルをロードして評価
model.load_state_dict(best_model_state)
print(f"ベスト検証精度: {best_val_acc:.2f}%")

# フォント設定を明示的に行い、警告を抑制
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# eval.pyを使用して評価
print("\nテストデータで評価中...")
from eval import evaluate_baseline_model
results = evaluate_baseline_model(model, test_loader, device=device, num_intervals=30)

print(f"\n=== 評価結果 ===")
print(f"  Accuracy: {results['accuracy']:.4f}")
print(f"  Precision: {results['precision']:.4f}")
print(f"  Recall: {results['recall']:.4f}")
print(f"  F1-score: {results['f1_score']:.4f}")
print(f"  Loss (CrossEntropyLoss): {results['loss']:.6f}")


In [ ]:
# 学習曲線を可視化（訓練損失のみ）
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

fig, ax = plt.subplots(1, 1, figsize=(8, 5))

ax.plot(train_losses, label='Train Loss', color='blue')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training Curve (Loss)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('baseline_training_curves.png', dpi=150, bbox_inches='tight')
print("学習曲線を 'baseline_training_curves.png' に保存しました")

plt.show()


In [ ]:
# モデルを保存
torch.save({
    'model_state_dict': model.state_dict(),
    'test_accuracy': test_acc,
    'results': results
}, 'baseline_model.pth')
print("モデルを 'baseline_model.pth' に保存しました")


## 📥 結果ファイルのダウンロード

以下のセルを実行して、結果ファイルをダウンロードできます：
- `baseline_model.pth` (学習済みモデル)
- `baseline_confusion_matrix.png` (混同行列)
- `baseline_training_curves.png` (学習曲線)


In [ ]:
# 結果ファイルのダウンロード
from google.colab import files

files.download('baseline_model.pth')
files.download('baseline_confusion_matrix.png')
files.download('baseline_training_curves.png')
print("✓ ダウンロード完了！")


## 代替方法: コードでアップロード（ドラッグ&ドロップが使えない場合）

上記のファイル確認でファイルが見つからない場合のみ、このセルを実行してください。


In [ ]:
# ファイルアップロード用（代替方法）
# ⚠️ ドラッグ&ドロップの方が速いので、そちらを推奨します

from google.colab import files

print("以下のファイルをアップロードしてください:")
print("1. baseline_dataset.pickle")
print("2. baseline_model.py")
print("3. evaluate_noise_detection.py")
print("\nアップロードを開始します...")

uploaded = files.upload()

print("\nアップロード完了！")
for filename in uploaded.keys():
    print(f"  ✓ {filename}")


In [ ]:
# baseline_model.py と evaluate_noise_detection.py をインポート
from baseline_model import SimpleCNN
from evaluate_noise_detection import evaluate_baseline_model, plot_confusion_matrix

print("✓ モジュールのインポート完了")


In [ ]:
# ハイパーパラメータ
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_EPOCHS = 10  # Google ColabのGPUで高速実行
NUM_CLASSES = 30

# DataLoaderの作成
train_dataset = PSDDataset(train_data, train_labels)
val_dataset = PSDDataset(val_data, val_labels)
test_dataset = PSDDataset(test_data, test_labels)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# モデルの作成
model = SimpleCNN(num_classes=NUM_CLASSES).to(device)

# モデルの初期化を確認
print("\n=== モデルの初期化確認 ===
model.eval()
with torch.no_grad():
    # ダミーデータでテスト
    dummy_input = torch.randn(1, 3000).to(device)
    dummy_output = model(dummy_input)
    print(f"  ダミー入力の形状: {dummy_input.shape}")
    print(f"  ダミー出力の形状: {dummy_output.shape}")
    print(f"  ダミー出力の範囲: [{dummy_output.min().item():.4f}, {dummy_output.max().item():.4f}]")
    print(f"  ダミー出力の平均: {dummy_output.mean().item():.4f}")
    print(f"  ダミー出力の標準偏差: {dummy_output.std().item():.4f}")
    print(f"  各クラスの出力: {dummy_output[0].tolist()}")
    print(f"  ソフトマックス後の確率: {torch.softmax(dummy_output[0], dim=0).tolist()}")
model.train()
print(f"モデルパラメータ数: {sum(p.numel() for p in model.parameters()):,}")

# 損失関数とオプティマイザ
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

In [ ]:
# 学習実行
import time

# 学習履歴
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
best_val_acc = 0
best_model_state = None

print("学習開始...")
print("=" * 60)
print(f"総エポック数: {NUM_EPOCHS}")

start_time = time.time()

for epoch in range(NUM_EPOCHS):
    epoch_start = time.time()
    
    # 学習
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # 検証
    val_loss, val_acc, _, _ = validate(model, val_loader, criterion, device)
    
    # 学習率の調整
    scheduler.step(val_loss)
    
    # 履歴の保存
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    # ベストモデルの保存
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict().copy()
        patience_counter = 0  # ベスト精度更新時にリセット
    else:
        patience_counter += 1  # ベスト精度が更新されなかった
    
    # Early Stoppingチェック
    if patience_counter >= patience:
        print(f"\n⚠ Early Stopping: {patience}エポック連続で改善なし。学習を停止します。")
        print(f"   ベスト検証精度: {best_val_acc:.2f}% (エポック {epoch+1-patience})")
        break
        best_val_acc = val_acc
        best_model_state = model.state_dict().copy()
    
    # ログ出力（5エポックごと、または最初と最後）
    if (epoch + 1) % 5 == 0 or epoch == 0 or (epoch + 1) == NUM_EPOCHS:
        epoch_time = time.time() - epoch_start
        elapsed_time = time.time() - start_time
        remaining_epochs = NUM_EPOCHS - (epoch + 1)
        estimated_remaining = (elapsed_time / (epoch + 1)) * remaining_epochs
        
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] ({epoch_time:.1f}秒)")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"  経過時間: {elapsed_time/60:.1f}分, 残り見積もり: {estimated_remaining/60:.1f}分")
        print()

total_time = time.time() - start_time
print(f"学習完了！総時間: {total_time/60:.1f}分 ({total_time/3600:.2f}時間)")